<a href="https://colab.research.google.com/github/hirajya/CCDEPLRL_EXERCISES_COM222/blob/main/Exercise6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 6

In [262]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [263]:
import numpy as np
import pandas as pd

path = "https://github.com/robitussin/CCDEPLRL_EXERCISES/blob/9b8ac1c5683abecc144f0af47eb7cda0688e12b7/dataset/reviews.json?raw=true"

dataset = pd.read_json(path)

In [264]:
dataset.head()

,review,rating
0,sir okay armygreen shorts nice,5
1,di pareha yong mga size nila may sobrang liit ...,5
2,super worth it ang ganda Sombra grabi order na...,5
3,ganda po salamat,5
4,maayos pagkadeliver maganda den sya,5


In [265]:
dataset['rating'].value_counts()

,count
rating,
1,299
5,249
4,249
3,123
2,81


In [266]:
dataset.rename(columns={'rating': 'sentiment'}, inplace=True)

def convert_rating_to_sentiment(rating):
  if rating >= 3:
    return 1  # Positive
  else:
    return 0  # Negative

dataset['sentiment'] = dataset['sentiment'].apply(convert_rating_to_sentiment)

dataset.head()
dataset.info()
dataset['sentiment'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     1001 non-null   object
 1   sentiment  1001 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


,count
sentiment,
1,621
0,380


In [267]:
dataset.head()

,review,sentiment
0,sir okay armygreen shorts nice,1
1,di pareha yong mga size nila may sobrang liit ...,1
2,super worth it ang ganda Sombra grabi order na...,1
3,ganda po salamat,1
4,maayos pagkadeliver maganda den sya,1


In [268]:
dataset['sentiment'].value_counts()

,count
sentiment,
1,621
0,380


## 1. Tokenize the data

In [269]:
vocab_size = 5000
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(dataset['review'])

## 2. Sequence the data

In [270]:
sequences = tokenizer.texts_to_sequences(dataset['review'])

In [271]:
max_length = 100
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

## 3. Pad the data

In [272]:
max_length = 50
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

## 4. Train a sentiment model

In [273]:
X = padded_sequences
y = np.array(dataset['sentiment'])

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=80, validation_data=(X_val, y_val))


Epoch 1/80


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.5652 - loss: 0.6824 - val_accuracy: 0.6219 - val_loss: 0.6688
Epoch 2/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6292 - loss: 0.6571 - val_accuracy: 0.6219 - val_loss: 0.6642
Epoch 3/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6168 - loss: 0.6592 - val_accuracy: 0.6219 - val_loss: 0.6572
Epoch 4/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6430 - loss: 0.6403 - val_accuracy: 0.6169 - val_loss: 0.6475
Epoch 5/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6494 - loss: 0.6284 - val_accuracy: 0.6318 - val_loss: 0.6304
Epoch 6/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6683 - loss: 0.6074 - val_accuracy: 0.6617 - val_loss: 0.6092
Epoch 7/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7123 - loss: 0.5770 - val_accuracy: 0.6866 - val_loss: 0.5809
Epoch 8/80
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7975 - loss: 0.5315 - val_accuracy: 0.7164 - val_loss: 0.5491
Ep

## Get files for visualing the network

In [274]:
# First get the weights of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)


(5000, 32)


In [275]:


import io

# Create the reverse word index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Write out the embedding vectors and metadata
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in reverse_word_index:
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()


In [276]:
# # Download the files
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download('vecs.tsv')
#   files.download('meta.tsv')

## 5. Predict sentiment with new reviews

In [280]:
new_reviews = [
    "Ang panget naman niyan!",
    "Ganda ng damit dito, sulit",
    "Saktoh lmngs",
]

new_sequences = tokenizer.texts_to_sequences(new_reviews)
new_padded = pad_sequences(new_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(new_padded)

for i, review in enumerate(new_reviews):
    sentiment = "Positive" if predictions[i] >= 0.5 else "Negative"
    print(f"Review: {review}\nSentiment: {sentiment} (Confidence: {predictions[i][0]:.2f})\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Review: Ang panget naman niyan!
Sentiment: Negative (Confidence: 0.07)

Review: Ganda ng damit dito, sulit
Sentiment: Positive (Confidence: 1.00)

Review: Saktoh lmngs
Sentiment: Positive (Confidence: 0.69)

